In [2]:
import json
from pedesis.shortcuts import get_settings, get_engine
from pedesis.station.controller import Station

In [3]:
station = Station(get_settings())
station.start_all_public_components()

In [4]:
engines = station._engines_name
engines

['multi_long_triple_ema_scalp',
 'scalp_short_macd',
 'swing_long_bollinger_bands',
 'swing_short_bollinger_bands']

In [5]:
macd = get_engine(engines[1])
macd

In [6]:
base_srls_configs = dict(
    calculators=macd._settings.installed_srls,
    timeframes=macd._settings.srl_timeframes,
    engine=macd._settings.engine_property
)
base_srls_configs

{'calculators': ['pedesis.components.srl_calculator.calculators.williams_fractal'],
 'timeframes': ['5m',
  '15m',
  '30m',
  '1h',
  '2h',
  '4h',
  '6h',
  '12h',
  '1d',
  '3d',
  '1w',
  '1M'],
 'engine': EngineProperty(name='scalp_short_macd', mode=<EngineMode.Scalp: 'scalp'>, native_broker='pedesis.components.broker.templates.okx', market='future')}

In [7]:
symbols = macd._get_symbols_router_list()
symbols

[app.scalp_short_macd.router.BTCUSDT,
 app.scalp_short_macd.router.ETHUSDT,
 app.scalp_short_macd.router.DOGEUSDT]

In [8]:
def make_srls_record(srls_configs):
    from pedesis.components.srl_calculator.models import SRLsModel, UniSymbol
    calcs = srls_configs['calculators']
    calcs.sort()
    tms = srls_configs['timeframes']
    tms.sort()
    uni_symbol_id = UniSymbol.get('id', symbol=srls_configs['symbol'].__name__)
    srls = SRLsModel(
        signal_engine_signature=srls_configs['engine'].srl_signature(),
        calculators=json.dumps(calcs),
        timeframes=json.dumps(tms),
        uni_symbol_id=uni_symbol_id
    ).save()
    srls_configs['engine'].add_srls_in_cache(uni_symbol_id, srls.id)
    return srls

In [9]:
btc_srls = base_srls_configs.copy()
btc_srls['symbol'] = symbols[0]
btc_srls

{'calculators': ['pedesis.components.srl_calculator.calculators.williams_fractal'],
 'timeframes': ['5m',
  '15m',
  '30m',
  '1h',
  '2h',
  '4h',
  '6h',
  '12h',
  '1d',
  '3d',
  '1w',
  '1M'],
 'engine': EngineProperty(name='scalp_short_macd', mode=<EngineMode.Scalp: 'scalp'>, native_broker='pedesis.components.broker.templates.okx', market='future'),
 'symbol': app.scalp_short_macd.router.BTCUSDT}

In [10]:
from pedesis.components.srl_calculator.maker import SRLsStarter
btc_starter = SRLsStarter(make_srls_record(btc_srls).get_configs())
btc_starter

In [13]:
def srls_start(srls_starter):
    from pedesis.components.srl_calculator.maker import get_srl, SRLConfigs
    from pedesis.logger import get_logger
    from pedesis.cache import get_redlock
    import itertools
    
    _logger = get_logger()
    _logger.info(f"SRLsStarter --start-- make SRL class for this SRLs: {srls_starter.configs.cache_uid}")
    srl_tasks = []
    for engine, uni_symbol_id, tm, calc in itertools.product([srls_starter.configs.engine], [srls_starter.configs.uni_symbol_id], srls_starter.configs.timeframes, srls_starter.configs.calculators):
        conf = SRLConfigs(
            engine=engine,
            uni_symbol_id=uni_symbol_id,
            timeframe=tm,
            calculator=calc,
        )
        _logger.info(f"SRLsStarter --start-- create SRL for {conf.cache_uid}.")
        with get_redlock(conf.get_redlock_key()):
            srl = get_srl(conf, log=_logger)
            srl_tasks.append(srl.make_task())
            _logger.info(f"SRLsStarter --end-- create SRL for {conf.cache_uid}.")
            srl.add_srls(srls_starter.configs.dbid)
        srls_starter.installed_srl[srl.configs.dbid] = srl._calculator_settings

In [14]:
srls_start(btc_starter)

In [15]:
btc_starter.installed_srl

{1: CalculatorTimeFrameSettings(timeframe='12h', srlevel_std_multiplier=1.05, data_settings=DataSettings(start_time=1609459200000.0, required_length=None, other=None), first_std_srl=0.025, other={'period': 10}),
 2: CalculatorTimeFrameSettings(timeframe='15m', srlevel_std_multiplier=1.05, data_settings=DataSettings(start_time=1609459200000.0, required_length=None, other=None), first_std_srl=0.025, other={'period': 20}),
 3: CalculatorTimeFrameSettings(timeframe='1M', srlevel_std_multiplier=1.05, data_settings=DataSettings(start_time=1609459200000.0, required_length=None, other=None), first_std_srl=0.025, other={'period': 2}),
 4: CalculatorTimeFrameSettings(timeframe='1d', srlevel_std_multiplier=1.05, data_settings=DataSettings(start_time=1609459200000.0, required_length=None, other=None), first_std_srl=0.025, other={'period': 7}),
 5: CalculatorTimeFrameSettings(timeframe='1h', srlevel_std_multiplier=1.05, data_settings=DataSettings(start_time=1609459200000.0, required_length=None, ot

In [17]:
from pedesis.components.srl_calculator.models import *

SRLineRaw.all(in_df=True)

,open,id,srl_id,timestamp,index_price,high,low,close,volume
0,41407.6,1,1,1.610107e+12,40657.125,42077.6,38596.0,40547.3,3577496.0
1,34221.1,2,1,1.610366e+12,33903.600,35601.7,30431.4,35360.2,7602135.0
2,37040.6,3,1,1.611922e+12,35840.950,38600.0,33500.0,34223.2,4271187.0
3,30651.2,4,1,1.611749e+12,30489.750,31675.2,29258.4,30374.2,3712610.0
4,30864.2,5,1,1.611274e+12,30880.350,32060.0,28860.0,31737.2,4537623.0
...,...,...,...,...,...,...,...,...,...
15906,20961.7,15907,12,1.655813e+12,21201.625,21749.2,20851.0,21244.6,5937205.0
15907,16211.7,15908,12,1.669637e+12,16204.550,16399.0,16000.0,16207.5,3924396.0
15908,16498.7,15909,12,1.672402e+12,16488.275,16581.0,16337.0,16536.4,1963644.0
15909,24576.0,15910,12,1.676549e+12,24751.250,25251.2,24290.3,24887.5,8384968.0


In [18]:
srlines_raw_stmt = SRLineRaw.get_select().where(SRLineRaw.srl_id.in_(list(btc_starter.installed_srl.keys())))
srlines_raw = SRLineRaw.query(srlines_raw_stmt, in_df=True)
srline_stmt = SRLine.get_select().where(SRLine.srline_raw_id.in_(list(srlines_raw.id)))
srlines = SRLine.query(srline_stmt, in_df=True)

In [19]:
srlines

,id,srls_id,srline_raw_id,srlevel_id


In [20]:
srlines_raw

,open,id,srl_id,timestamp,index_price,high,low,close,volume
0,41407.6,1,1,1.610107e+12,40657.125,42077.6,38596.0,40547.3,3577496.0
1,34221.1,2,1,1.610366e+12,33903.600,35601.7,30431.4,35360.2,7602135.0
2,37040.6,3,1,1.611922e+12,35840.950,38600.0,33500.0,34223.2,4271187.0
3,30651.2,4,1,1.611749e+12,30489.750,31675.2,29258.4,30374.2,3712610.0
4,30864.2,5,1,1.611274e+12,30880.350,32060.0,28860.0,31737.2,4537623.0
...,...,...,...,...,...,...,...,...,...
15906,20961.7,15907,12,1.655813e+12,21201.625,21749.2,20851.0,21244.6,5937205.0
15907,16211.7,15908,12,1.669637e+12,16204.550,16399.0,16000.0,16207.5,3924396.0
15908,16498.7,15909,12,1.672402e+12,16488.275,16581.0,16337.0,16536.4,1963644.0
15909,24576.0,15910,12,1.676549e+12,24751.250,25251.2,24290.3,24887.5,8384968.0
